In [1]:
from server import LinkedViewsServer
data_folder = "/scratch/visual/bzfharth/in-silico-install-dir/project_src/in_silico_framework/getting_started/linked-views-example-data/backend_data_2023-06-22"

server = LinkedViewsServer(data_folder)
server.start(5000)
#server.stop()

start read: simulation_samples_original.csv
complete read
start read: simulation_samples.csv
complete read
server is running at port 5000
logs are written to /tmp/linked-views-server.log
set data:        server.set_data(vaex_df)
stop server:     server.stop()


In [2]:
import vaex
import vaex.ml
import util

def append_PCA(df, columns, descriptor, n_components = 2):
    pca = vaex.ml.PCA(features=columns, n_components=n_components)    
    df_transformed = pca.fit_transform(df)
    for component_idx in range(0, n_components):
        df_transformed.rename(f"PCA_{component_idx}", f"{descriptor}-{component_idx+1}")
    return df_transformed

In [3]:
filename = "/scratch/visual/bzfharth/in-silico-install-dir/project_src/in_silico_framework/getting_started/linked-views-example-data/backend_data_2023-06-22/simulation_samples.csv"
df = vaex.from_csv(filename, copy_index=False)

df_extended = append_PCA(df, util.getParamsCols(), "PCA-ephys")

server.set_data(df_extended)


In [6]:
if("global" in server.selections):
    columns = server.selections["global"]["columns"] # ['ephys.CaDynamics_E2_v2.apic.decay','ephys.NaTa_t.axon.gNaTa_tbar']
    col1 = columns[0]
    col2 = columns[1]

    print(columns)

    ranges = server.selections["global"]["bin_ranges"]

    server.vaex_df.select_nothing(name="selection_global")
    for idx, range_i in enumerate(ranges):    
        limit_i = [(range_i[0][0], range_i[0][1]), (range_i[1][0], range_i[1][1])]        
        server.vaex_df.select_rectangle(server.vaex_df[col1], server.vaex_df[col2], limit_i, name="selection_global", mode="or")
        #print(limit_i)

    df_selected_indices = server.vaex_df.evaluate(server.vaex_df["Unnamed: 0"], selection="selection_global")

    print("num selected", df_selected_indices.shape)
    print("indices", df_selected_indices)

['PCA-ephys-1', 'PCA-ephys-2']
num selected (259,)
indices [  12   48   94  170  201  256  355  411  442  461  507  525  545  579
  652  830  842  851  915  925  940 1023 1269 1288 1445 1454 1465 1468
 1493 1549 1562 1568 1780 1830 1848 1913 1957 1964 1997 2002 2004 2014
 2073 2149 2154 2169 2244 2334 2369 2373 2399 2429 2437 2557 2558 2600
 2693 2746 2815 2825 2826 2827 2893 2928 2950 2959 2975 2987 2993 2997
 3006 3034 3037 3102 3111 3122 3233 3366 3429 3476 3555 3719 3722 3759
 3809 3823 3826 3844 3855 3903 3918 3944 3960 3994 4027 4029 4106 4124
 4132 4150 4239 4251 4283 4307 4352 4395 4501 4543 4673 4691 4693 4800
 4807 4838 4876 4932 4948 4981 5018 5031 5069 5126 5143 5154 5157 5196
 5224 5262 5289 5306 5321 5331 5342 5416 5478 5494 5551 5561 5578 5582
 5615 5638 5670 5696 5735 5767 5837 5876 5920 5940 5986 5996 6052 6088
 6147 6161 6168 6200 6210 6340 6376 6396 6400 6421 6425 6561 6609 6626
 6630 6666 6709 6713 6722 6744 6796 6803 6874 6954 7042 7172 7199 7205
 7266 7322 7350 73